In [1]:
import pandas as pd
import joblib
from xgboost import XGBClassifier
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../data/res_purchase_card_fiscal_year_2014_3pcd-aiuu.csv')
data.head(5)

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC)
0,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,GENERAL PURCHASE,890.00,NACAS,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,CHARITABLE AND SOCIAL SERVICE ORGANIZATIONS
1,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,ROOM CHARGES,368.96,SHERATON HOTEL,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,SHERATON
2,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,J,GENERAL PURCHASE,165.82,SEARS.COM 9300,07/29/2013 12:00:00 AM,07/31/2013 12:00:00 AM,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...
3,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,T,GENERAL PURCHASE,96.39,WAL-MART #0137,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"GROCERY STORES,AND SUPERMARKETS"
4,201307,1000,OKLAHOMA STATE UNIVERSITY,Mauro-Herrera,M,HAMMERMILL COPY PLUS COPY EA,125.96,STAPLES DIRECT,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"STATIONERY, OFFICE SUPPLIES, PRINTING AND WRIT..."


### Feature Extraction

With the given data, the following features can be extracted:
1) Weekday on which the transaction took place
2) Month in which this transaction took place
3) Time interval(K) in between transactions for each cardholder
4) There are some transactions where the transaction consist of negative amount that we came across in EDA , this indicates a refund. These can be used as our labels.

In [3]:
df = data.copy()


df["name"] = df["Cardholder First Initial"].astype(str).str.cat(df["Cardholder Last Name"].astype(str), sep="_")
df = df[["name", "Transaction Date", "Amount"]]
df.columns = ["name", "transaction_date", "amount_spend"]
df["transaction_date"] = pd.to_datetime(df["transaction_date"])
df["weekday"] = df["transaction_date"].dt.weekday
df["month"] = df["transaction_date"].dt.month
df["k"] = df.sort_values(["name", "transaction_date"]).groupby("name")["transaction_date"].diff().dt.days
df.dropna(subset=["k"], inplace=True)

df["y_label"] = df["amount_spend"].map(lambda x: 1 if x >0 else 0)
df["encd_name"] = df.name.astype('category').cat.codes
df

,name,transaction_date,amount_spend,weekday,month,k,y_label,encd_name
0,C_Mason,2013-07-30,890.00,1,7,3.0,1,815
1,C_Mason,2013-07-30,368.96,1,7,0.0,1,815
2,J_Massey,2013-07-29,165.82,0,7,4.0,1,2192
3,T_Massey,2013-07-30,96.39,1,7,6.0,1,4722
4,M_Mauro-Herrera,2013-07-30,125.96,1,7,1.0,1,3335
...,...,...,...,...,...,...,...,...
442453,e_Cardholder,2014-06-20,992.00,4,6,0.0,1,5032
442454,e_Cardholder,2014-06-20,2415.00,4,6,0.0,1,5032
442455,e_Cardholder,2014-06-20,177.50,4,6,0.0,1,5032
442456,e_Cardholder,2014-06-19,32457.00,3,6,0.0,1,5032


In [4]:
df[df.y_label==0].groupby("name").agg({"name": "count"}).describe()

,name
count,3046.000000
mean,4.724885
std,13.117249
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,487.000000


In [5]:
df[df.y_label==1].groupby("name").agg({"name": "count"}).describe()

,name
count,5019.000000
mean,84.250448
std,238.335010
min,1.000000
25%,13.000000
50%,36.000000
75%,90.000000
max,9649.000000


In [6]:
df.name.unique().shape

(5036,)

In [7]:
df.shape

(437245, 8)

In [8]:
features_col = ["amount_spend", "weekday", "month", "k"]
X = df[features_col]
Y = df["y_label"]

corr = X.corr()
corr.style.background_gradient(cmap='coolwarm')

,amount_spend,weekday,month,k
amount_spend,1.000000,-0.004056,0.000504,0.000016
weekday,-0.004056,1.000000,-0.020389,-0.026957
month,0.000504,-0.020389,1.000000,-0.022340
k,0.000016,-0.026957,-0.022340,1.000000


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=df["y_label"])

In [10]:

X_train

,amount_spend,weekday,month,k
441032,178.52,2,4,0.0
372101,433.28,3,5,3.0
210644,116.96,4,8,1.0
410329,101.60,1,6,0.0
83164,17.10,2,9,0.0
...,...,...,...,...
188777,12.27,6,11,0.0
169989,40.00,0,8,0.0
317906,8.97,3,3,1.0
127686,436.00,6,8,1.0


In [11]:
Counter(y_test)

Counter({1: 84571, 0: 2878})

In [12]:
# Define and train the XGBoostClassifier model
model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_train)
roc_auc_score(y_train,y_pred, average="weighted")

C:\Users\sohai\miniconda3\envs\one_by_zero_submission_2\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:26:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


1.0

In [13]:
X_train.loc[0]

amount_spend    890.0
weekday           1.0
month             7.0
k                 3.0
Name: 0, dtype: float64

In [17]:
int(model.predict(X_train.iloc[[0]])[0])

1

In [19]:
import json

json.dumps({"1":int(model.predict(X_train.iloc[[0]])[0])})

'{"1": 1}'

In [15]:
joblib.dump(model, "../models/classification_model.pkl")
w = joblib.load( "../models/classification_model.pkl")

In [16]:
! python --version

Python 3.6.13 :: Anaconda, Inc.
